In [1]:
from hdbscan import HDBSCAN
import nltk
from nltk.corpus import verbnet as vn
import pandas as pd
from sklearn.cluster import SpectralClustering, MeanShift
from sklearn.decomposition import PCA
#from sklearn.manifold import TSNE
from sklearn.metrics import f1_score, adjusted_rand_score, adjusted_mutual_info_score, v_measure_score
from sklearn.metrics import fowlkes_mallows_score
from umap import UMAP


from cluster import ClusterVerbs
from eval_tensor import VerbTensorEvaluator

import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (8, 4)

Populating the interactive namespace from numpy and matplotlib


# Compared otVerbNet

In [57]:
def compare_to_verbnet(non_negative=False, decomp_algo='tucker', rank=64, cutoff=100000, weight='npmi',
                       min_cluster_size=50, min_samples=5):
    evalor = VerbTensorEvaluator(non_negative=non_negative, decomp_algo=decomp_algo, 
                                 rank=rank, cutoff=cutoff, weight=weight)
    evalor.load_embeddings()
    df = pd.DataFrame(evalor.index['ROOT'], columns=['verb'])
    df['vnet'] = df.verb.apply(lambda verb: vn.classids(lemma=verb))
    mapper = UMAP(n_neighbors=30, min_dist=0.0, n_components=10, metric='cosine')
    lowdim_vecs = mapper.fit_transform(evalor.decomped_tns.factors[1])
    clusser = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples)
    df['cluster'] = clusser.fit_predict(lowdim_vecs)
    return df, adjusted_rand_score(df.vnet.astype(str), df.cluster.astype(str))

In [56]:
print(pd.DataFrame(
    [(size, ', '.join(df[df.cluster==i].verb.values))
     for i, size in df.groupby('cluster').size().sort_values(ascending=False).head(20).to_dict().items()],
    columns=['n_verbs', 'verbs']).set_index('n_verbs').to_latex())

\begin{tabular}{ll}
\toprule
{} &                                              verbs \\
n\_verbs &                                                    \\
\midrule
1132    &  have, take, give, show, read, include, keep, p... \\
315     &  hurt, wake, inspire, bother, surprise, satisfy... \\
246     &  gon, wan, subscribe, reside, stumble, behave, ... \\
114     &  eat, drink, cook, burn, taste, wash, pour, pac... \\
103     &  break, fit, draw, catch, shoot, extend, beat, ... \\
92      &  provide, create, offer, pay, spend, build, con... \\
92      &  receive, support, seek, accept, sign, obtain, ... \\
80      &  believe, hope, agree, decide, report, plan, ex... \\
72      &  happen, lead, apply, occur, exist, tend, resul... \\
68      &  stop, cause, cover, manage, face, prevent, avo... \\
64      &  live, talk, stand, die, return, pass, walk, si... \\
62      &  leave, open, enter, visit, close, fill, fix, r... \\
58      &  be, get, say, make, go, use, think, know, see,... \\
54    

In [32]:
weights = ["iact" ,"iact_sali" ,"ldice" ,"ldice_sali" ,"log_freq" ,"niact" ,"npmi" ,"pmi" ,"pmi_sali"]

cutoffs = []
for i in [1, 2, 3, 5]:
    cutoffs += list(i*10**np.arange(2, 9))

def eval_agains_verbnet_for(algo='tucker'):    
    data = []
    for non_negative in range(2):
        for cutoff in cutoffs:
            for exp in range(10):
                rank = 2**exp
                for weight in weights:
                    try:
                        _, score = compare_to_verbnet(non_negative=non_negative, decomp_algo=algo,
                                                   rank=rank, cutoff=cutoff, weight=weight)
                        record = (cutoff, rank, weight, non_negative, algo, score)
                        logging.info((record))
                        data.append(record)
                    except FileNotFoundError:
                        pass
                    except AttributeError as e:
                        logging.warning((record, e))
                    except ValueError:
                        logging.warning(record)
    df = pd.DataFrame(data, columns=['cutoff', 'rank', 'weight', 'non_negative', 'algo', 'corr'])
    return df.sort_values('corr', ascending=False)

In [64]:
%time score_df = eval_agains_verbnet_for()

/home/makrai/tool/python/vtensor/lib/python3.7/site-packages/umap/umap_.py:126: UserWarning: A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 340 edges.
It has only fully disconnected 4 vertices.
Use umap.utils.disconnected_vertices() to identify them.
  f"A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.\n"


CPU times: user 54min 53s, sys: 24.6 s, total: 55min 17s
Wall time: 6min 46s


In [37]:
score_df.sort_values('corr', ascending=False)

,cutoff,rank,weight,non_negative,algo,corr
31,50000,32,log_freq,0,tucker,0.022485
3,100000,64,iact,0,tucker,0.021839
40,50000,64,pmi_sali,0,tucker,0.020478
9,100000,64,npmi,0,tucker,0.018066
1,1000,4,log_freq,0,tucker,0.015387
...,...,...,...,...,...,...
53,300000,64,log_freq,1,tucker,-0.012484
59,500000,64,log_freq,1,tucker,-0.015166
60,500000,64,npmi,1,tucker,-0.017200
26,300000,64,log_freq,0,tucker,-0.018261


In [34]:
df.groupby('cluster').size().sort_values(ascending=False).head(10)

cluster
-1     547
 29    100
 21     98
 30     94
 45     88
 25     81
 16     77
 5      71
 19     70
 4      69
dtype: int64

In [ ]:
#df.groupby(df.vnet.astype(str)).size().sort_values(ascending=False).head(10)

In [ ]:
ser_clust = df.groupby(df.cluster).size().sort_values(ascending=False).values
plt.plot(ser_clust + 1)
plt.xscale('log')
plt.yscale('log')

In [ ]:
ser_class = df.groupby(df.vnet.astype(str)).size().sort_values(ascending=False)
plt.plot(ser_class+1)
plt.xscale('log')
plt.yscale('log')

In [ ]:
df1 = df[(df.vnet.str.len()!=0)&(df.cluster!=-1)]
df1.groupby(['cluster', df1.vnet.astype(str)]).size().sort_values(ascending=False).head(20)

In [ ]:
adjusted_mutual_info_score(df.vnet.astype(str), df.cluster.astype(str))

In [ ]:
fowlkes_mallows_score(df.vnet.astype(str), df.cluster.astype(str))

In [ ]:
f1_score(df.vnet.apply(lambda l: 'amuse-31.1' in l), df.cluster==14)

In [ ]:
df = df.reset_index()

In [ ]:
plt.rcParams['figure.figsize'] = [16, 8]

In [ ]:
plt.scatter(*verb_mx.T[0:], s=3, c=df.index)
plt.colorbar()

In [ ]:
_ = plt.hist2d(df.index, df.vnet.str.len())

In [ ]:
def least_class(classes):
    if classes:
        return min([int(class_.split('-')[1].split('.')[0]) for class_ in classes])
    else:
        return np.nan

In [ ]:
df['least_class'] = df.vnet.apply(least_class)

In [ ]:
df = df.fillna(df.least_class.max()+1)

In [ ]:
df[df.vnet.str.len()==0].head()

In [ ]:
ser = df.groupby('least_class').size()

In [ ]:
ser.name = 'lclass_size'

In [ ]:
df = df.set_index('least_class').join(ser)

In [ ]:
df[df.least_class==9].head()

In [ ]:
part = (df.lclass_size > 50).values
plt.scatter(*verb_mx[part].T, s=5, c=df.least_class[part])#==9)
plt.colorbar()